<a href="https://colab.research.google.com/github/Anonymous1846/Blood-cancer-subtype-classification-using-DCNN/blob/master/Blood_Cancer_Subtype_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the required dependencies !**

In [ ]:
'''
  The Keras libraray provides APIs for layering and image data augmentation !
'''
from keras.layers import MaxPooling2D,Conv2D,Dropout,Dense,Flatten,BatchNormalization,MaxPool2D,AveragePooling2D 
from keras.applications.imagenet_utils import decode_predictions  
from keras.callbacks import ModelCheckpoint,EarlyStopping 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop,Adam    
from sklearn.metrics import classification_report    
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19                            
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix                           
from sklearn.metrics import roc_auc_score    
from sklearn.metrics import roc_curve    
from sklearn.metrics import auc 
import matplotlib.pyplot as plt                                                   
import tensorflow as tf
from cv2 import imread
import numpy as np
import pickle                                                                     
import keras
import os 

<module 'tensorflow' from '/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py'>


# **Loading the dataset from the google drive**

In [ ]:
'''
This is an optional block of code which can be executed if the 
google drive needs to be mounted on different account
'''

from google.colab import drive
drive.mount('/content/drive',force_remount=False)